# Unit decorators

### cost decorator

The cost decorator adds a free on board purchase cost based on the exponential scale up equation:

$$New\ cost = N \cdot cost \bigg(\frac{CE_{new}}{CE}\bigg) \bigg(\frac{S_{new}}{N \cdot S}\bigg)^{n}$$
    
$$Electricity\ rate = kW \bigg(\frac{S_{new}}{S}\bigg)$$

$$N = ceil \bigg( \frac{S_{new}}{ub} \bigg)$$

$N\text{: Number of units}$

$ub\text{: Upper bound of size}$

$CE_{new}\text{: New Chemical Engineering Plant Cost Index}$

$CE\text{: Chemical Engineering Plant Cost Index}$

$S_{new}\text{: New size value}$

$S\text{: Original size value}$

$cost\text{: Free on board purchase cost at size S}$

$kW\text{: Electricity rate in kW at size S}$

$n\text{: Exponential factor}$

$lifetime\text{: Number of operating years until equipment needs to be replaced}$

Create a Shredder Unit subclass for sugar cane with the following exponential scale up factors [[1]](#References):

* Chemical Engineering Plant Cost Index: 567.3
* Size: 500,000 kg/hr flow rate
* Purchase cost: $2,500,000
* Electricity rate: 3,000 kW
* Exponential factor: 0.6
* Bare module factor: 1.39
* Equipment lifetime: 30

Additionally, include a bare module factor of 1.39 as an approximation for this example [[2]](#References).

In [1]:
import biosteam as bst
from biosteam.units.decorators import cost

# Set up thermo property package
chemicals = bst.Chemicals(['Water', 'Ethanol'])
SugarCane = bst.Chemical.blank('SugarCane', phase_ref='s')
SugarCane.default()
chemicals.append(SugarCane)
bst.settings.set_thermo(chemicals)

# For year 2018
bst.CE = 603.1 

# basis will be the total flow rate in kg/hr
@cost('Flow rate', units='kg/hr', cost=2.5e6, CE=567.3,
      n=0.6, S=500e3, kW=3000, BM=1.39, lifetime=30)
class Shredder(bst.Unit): pass
# Units without a `_run` method assume one input and output stream
# that share the same flow rates and conditions

### Test a Shreadder object ###

# Display flow rate in kg/hr
bst.Stream.display_units.flow = 'kg/hr'

feed = bst.Stream(SugarCane=1e6, units='kg/hr')
shredder = Shredder(ins=feed)
shredder.simulate()

shredder.show()
shredder.results()

Shredder: U1
ins...
[0] s1
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow (kg/hr): SugarCane  1e+06
outs...
[0] s2
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow (kg/hr): SugarCane  1e+06


Shredder                        Units       U1
Power               Rate           kW    6e+03
                    Cost       USD/hr      469
Design              Flow rate   kg/hr    1e+06
Purchase cost       Shredder      USD 4.03e+06
Total purchase cost               USD 4.03e+06
Utility cost                   USD/hr      469

Cost data is stored as CostItem objects in the `cost_items` dictionary:

In [2]:
Shredder.cost_items

{'Shredder': <CostItem: Flow rate (kg/hr)>}

In [3]:
Shredder.cost_items['Shredder']

CostItem: Flow rate (kg/hr)
 S     5e+05
 CE    567
 cost  2.5e+06
 n     0.6
 kW    3e+03



Bare module factors and equipment lifetimes are stored separately:

In [4]:
Shredder._F_BM_default

{'Shredder': 1.39}

In [5]:
Shredder._default_equipment_lifetime

{'Shredder': 30}

Any option can be changed:

In [6]:
Shredder.cost_items['Shredder'].cost = 3e6 # Change base cost
# This also works:
# Shredder.cost_items['Shredder']['cost'] = 3e6
shredder.simulate()
shredder.results()

Shredder                        Units       U1
Power               Rate           kW    6e+03
                    Cost       USD/hr      469
Design              Flow rate   kg/hr    1e+06
Purchase cost       Shredder      USD 4.83e+06
Total purchase cost               USD 4.83e+06
Utility cost                   USD/hr      469

In [7]:
Shredder.cost_items['Shredder'].ub = 6e5 # Change size upper bound
shredder.simulate()
shredder.results()

Shredder                        Units       U1
Power               Rate           kW    6e+03
                    Cost       USD/hr      469
Design              Flow rate   kg/hr    1e+06
                    #Shredder                2
Purchase cost       Shredder      USD 6.38e+06
Total purchase cost               USD 6.38e+06
Utility cost                   USD/hr      469

Note: It is also possible to decorate a Unit subclass multiple times to add multiple cost items.

It is also possible to extend the decorated cost. In this next example we will add an agitator to a flash vessel with the following exponential scale up factors [[3]](#References):

* Chemical Engineering Plant Cost Index: 522
* Size: 252,891 kg/hr flow rate
* Purchase cost: $90,000
* Electricity rate: 170 kW
* Exponential factor: 0.5
* Bare module factor: 1.5
* Lifetime: Number of biorefinery operating years

In [8]:
@cost('Flow rate', 'Agitator', units='kg/hr',
      cost=90e3, S=252891, kW=170, CE=522, n=0.5, BM=1.5)
class FlashWithAgitator(bst.Flash):
    
    def _design(self):
        super()._design()
        self._decorated_design()
    
    def _cost(self):
        # Run flash cost algorithm
        super()._cost()
        
        # Save vacuum pump rate to add it back later.
        if self.P < 101325:
            rate = self.power_utility.rate # Has a vacuum pump
        else:
            rate = 0 # No vacuum
        
        # Run decorated cost algorithm
        self._decorated_cost()
        
        if rate:
            # Agitator rate replaced vacuum pump rate
            # so add both and set power utility
            self.power_utility(self.power_utility.rate + rate)
        
# Test
F1 = FlashWithAgitator('F1', bst.Stream('feed', Water=800, Ethanol=500, T=350),
                       V=0.5, P=101325)
F1.simulate()
F1.results()

Flash with agitator                                   Units            F1
Power               Rate                                 kW          25.2
                    Cost                             USD/hr          1.97
Low pressure steam  Duty                              kJ/hr      2.81e+07
                    Flow                            kmol/hr           725
                    Cost                             USD/hr           172
Design              Vessel type                                  Vertical
                    Length                               ft          13.5
                    Diameter                             ft           8.5
                    Weight                               lb      8.61e+03
                    Wall thickness                       in         0.438
                    Flow rate                         kg/hr      3.74e+04
                    Vessel material                          Carbon steel
Purchase cost       Heat exchanger - Floating head      USD      3.57e+04
                    Vertical pressure vessel            USD      4.35e+04
                    Platform and ladders                USD      1.26e+04
                    Agitator                            USD         4e+04
Total purchase cost                                     USD      1.32e+05
Utility cost                                         USD/hr           174

### References

<a id='References'></a>

1. Huang, H., Long, S., & Singh, V. (2016) "Techno-economic analysis of biodiesel and ethanol co-production from lipid-producing sugarcane" Biofuels, Bioproducts and Biorefining, 10(3), 299–315. https://doi.org/10.1002/bbb.1640

2. Seider, W. D., Lewin,  D. R., Seader, J. D., Widagdo, S., Gani, R., & Ng, M. K. (2017). Product and Process Design Principles. Wiley. Cost Accounting and Capital Cost Estimation (Chapter 16)

3. Humbird, D., Davis, R., Tao, L., Kinchin, C., Hsu, D., Aden, A., Dudgeon, D. (2011). Process Design and Economics for Biochemical Conversion of Lignocellulosic Biomass to Ethanol: Dilute-Acid Pretreatment and Enzymatic Hydrolysis of Corn Stover (No. NREL/TP-5100-47764, 1013269). https://doi.org/10.2172/1013269
